In [32]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# -------------------------
# DEVICE
# -------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# MODEL ARCHITECTURE
# -------------------------
class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [33]:
MODEL_PATH = "cnn_pipeline_model.pth"  # your trained weights

checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)

model = CNNModel(num_classes=2).to(DEVICE)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

print("✅ Model loaded successfully")


✅ Model loaded successfully


In [34]:
IMG_SIZE = (224, 224)

preprocessing_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()   # NO normalization (same as training)
])


In [35]:
def predict_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img_tensor = preprocessing_transform(img).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        outputs = model(img_tensor)           # shape: [1, 2]
        probs = torch.softmax(outputs, dim=1)
        pred_class = probs.argmax(dim=1).item()
        confidence = probs[0, pred_class].item()

    label_map = {
        1: "Non-defective",
        0: "Defective"
    }

    return label_map[pred_class], confidence


In [52]:
image_path = r"C:\Users\maila\Desktop\Defect_Detection\Renamed_Not_OK\3000.jpg"   # change this

label, conf = predict_image(image_path)

print(f"Prediction : {label}")
print(f"Confidence : {conf:.2%}")


Prediction : Defective
Confidence : 99.97%
